In [1]:
import pandas as pd
from sklearn.preprocessing import Imputer
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_data = pd.read_csv('train.csv')
test_X = pd.read_csv('test.csv')

In [3]:
# Drop all rows with missing data on column 'SalePrice'
train_data = train_data.dropna(axis=0, how='any',subset=['SalePrice'])

In [4]:
train_y = train_data.SalePrice
train_X = train_data.drop(['SalePrice'],axis=1).select_dtypes(exclude=['object'])
test_X = test_X.select_dtypes(exclude=['object'])

## Handling Missing Data

In [5]:
def missing_data_marker(df):
    cols_with_missing = (col for col in df.columns if df[col].isnull().any())
    for col in cols_with_missing:
        df[col+'_was_missing'] = df[col].isnull()
    return df

In [6]:
# train_X = missing_data_marker(train_X)
# test_X = missing_data_marker(test_X)

### Making Pipelines & Finding best model and params using Cross Validation

In [7]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import Imputer
from xgboost import XGBRegressor
learning_rate = 0.01
for n_estimators in range(1600,1701,30):
    
    my_pipeline = make_pipeline(Imputer(), XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate, n_jobs=4))

    scores = cross_val_score(my_pipeline, train_X, train_y, scoring='neg_mean_absolute_error',cv=5)
    print('CrosValidation MAE of %d: %2f' %(i,-1 * scores.mean()))

CrosValidation MAE of 1600: 16252.468638
CrosValidation MAE of 1630: 16252.793196
CrosValidation MAE of 1660: 16248.478307
CrosValidation MAE of 1690: 16249.049307


In [8]:
my_model = XGBRegressor(n_estimators=1680, learning_rate=0.01, n_jobs=4)
my_model.fit(train_X,train_y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1680,
       n_jobs=4, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

### Partial Dependence Plot

In [9]:
predictions= my_model.predict(test_X)
predictions

array([125347.52, 155382.47, 178375.28, ..., 184897.27, 120127.75,
       239036.52], dtype=float32)